# Overview

Flask-RESTx is a powerful extension of Flask framework. Complete toolkit for building modular, well-documented, and secure APIs. 

Here’s a summary of the key components:

1. **Namespaces**: Organize your API.
2. **Resources**: Define the HTTP methods for endpoints.
3. **Models**: Manage input/output data and validation.
4. **Swagger UI**: Automatically generate interactive documentation.
5. **Error Handling**: Uniform error responses for easy debugging.
6. **Security**: Protect your endpoints using JWT tokens or other authentication methods.

# Installation

In [2]:
%pip install flask-restx

Note: you may need to restart the kernel to use updated packages.


# Setting Up a Basic Flask-RESTx Application

In [3]:
from flask import Flask
from flask_restx import Api, Resource

app = Flask(__name__)
api = Api(app)

@api.route('/hello')
class HelloWorld(Resource):
    def get(self):
        return {'hello': 'world'}

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\TFS\Study\learning-python\.venv\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Breakdown of Key Components:

1. **Flask**: Flask provides the web framework.
2. **Api Object**: `Api` is the base object that manages the API. It provides routing for your resources and serves as the foundation for building the API.
3. **Resource**: A `Resource` represents an entity on the web, like a user or a task. It contains methods such as `get()`, `post()`, `put()`, `delete()` to handle various HTTP requests.

# Using Namespaces in Flask-RESTx

**Namespaces** allow you to organize your API logically and avoid route conflicts. This is useful when building larger applications with multiple modules.

In [4]:
from flask_restx import Namespace, Resource

# Define a new namespace
user_ns = Namespace('users')

@user_ns.route('/')
class UserList(Resource):
    def get(self):
        return {'message': 'List of users'}

@user_ns.route('/<int:id>')
class User(Resource):
    def get(self, id):
        return {'message': f'User {id}'}

api.add_namespace(user_ns, '/api')

# Creating a Complete CRUD API with Flask-RESTx

In [6]:
from flask_restx import fields

# Define a model
user_model = api.model('User', {
    'id': fields.Integer,
    'name': fields.String,
    'age': fields.Integer
})

users = []

@user_ns.route('/')
class UserList(Resource):
    @api.marshal_with(user_model)
    def get(self):
        return users

    @api.expect(user_model)
    def post(self):
        new_user = api.payload
        users.append(new_user)
        return {'message': 'User added'}, 201

@user_ns.route('/<int:id>')
class User(Resource):
    @api.marshal_with(user_model)
    def get(self, id):
        user = next((u for u in users if u['id'] == id), None)
        if user:
            return user
        return {'message': 'User not found'}, 404

    def delete(self, id):
        global users
        users = [u for u in users if u['id'] != id]
        return {'message': 'User deleted'}, 204

api.add_namespace(user_ns, '/api')

1. `@api.marshal_with`: Defines the structure of the response data.
2. `@api.expect`: Validates the input data.
3. `HTTP Status Codes`: Each response returns an appropriate HTTP status (201 for resource creation, 204 for deletion, etc.).

# Automatic Swagger Documentation
Flask-RESTx automatically generates Swagger UI for the API. Swagger is an interface that allows developers and users to interact with the API via the browser.

Simply run the application and go to: http://127.0.0.1:5000/

# Error Handling in Flask-RESTx
Flask-RESTx makes error handling simple and consistent by allowing custom error responses.
This ensures that every error follows a standard structure, making it easier to debug and communicate issues to the user.

In [7]:
@user_ns.errorhandler(ValueError)
def handle_value_error(error):
    return {'message': str(error)}, 400

# Flask-RESTx and Security
Flask-RESTx supports authentication and authorization to secure your API:

1. **Token-based Authentication**: JWT (JSON Web Token) is a common method.
2. **Role-based Access Control (RBAC)**: Grant access to specific endpoints based on user roles.

In [8]:
from flask_jwt_extended import JWTManager

app.config['JWT_SECRET_KEY'] = 'secret'
jwt = JWTManager(app)

@api.route('/secure')
class SecureResource(Resource):
    @jwt_required()
    def get(self):
        return {'message': 'This is a secured route.'}

ModuleNotFoundError: No module named 'flask_jwt_extended'